<a href="https://colab.research.google.com/github/Priyatham10/NLP_work_sample/blob/master/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from utils import *
from IPython.display import display,HTML

## Text preprocessing

### Tokenization

### Word tokenization with fastai

In [5]:
from fastai2.text.all import *
path = untar_data(URLs.IMDB)

In [0]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [7]:
files

(#100000) [Path('/root/.fastai/data/imdb/train/neg/12159_1.txt'),Path('/root/.fastai/data/imdb/train/neg/3191_1.txt'),Path('/root/.fastai/data/imdb/train/neg/4982_4.txt'),Path('/root/.fastai/data/imdb/train/neg/5761_4.txt'),Path('/root/.fastai/data/imdb/train/neg/2844_4.txt'),Path('/root/.fastai/data/imdb/train/neg/1600_1.txt'),Path('/root/.fastai/data/imdb/train/neg/2519_2.txt'),Path('/root/.fastai/data/imdb/train/neg/1147_4.txt'),Path('/root/.fastai/data/imdb/train/neg/1374_1.txt'),Path('/root/.fastai/data/imdb/train/neg/7883_1.txt')...]

In [8]:
txt = files[0].open().read(); txt[:75]

"i can't believe i actually watched this but i guess i just wanted to know i"

**Here the `WordTokenizer()` uses Spacy library to tokenize the tokenization process.**

In [9]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#273) ['i','ca',"n't",'believe','i','actually','watched','this','but','i','guess','i','just','wanted','to','know','if','this','movie','would','get','unintentionally','funnier','and','funnier','towards','the','end','.','and'...]


In [10]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [11]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#276) ['xxbos','i','ca',"n't",'believe','i','actually','watched','this','but','i','guess','i','just','wanted','to','know','if','this','movie','would','get','unintentionally','funnier','and','funnier','towards','the','end','.','and'...]


In [12]:
defaults.text_proc_rules

[<function fastai2.text.core.fix_html>,
 <function fastai2.text.core.replace_rep>,
 <function fastai2.text.core.replace_wrep>,
 <function fastai2.text.core.spec_add_spaces>,
 <function fastai2.text.core.rm_useless_spaces>,
 <function fastai2.text.core.replace_all_caps>,
 <function fastai2.text.core.replace_maj>,
 <function fastai2.text.core.lowercase>]

In [13]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index'...]"

### Subword tokenization

In [0]:
txts = L(o.open().read() for o in files[:2000])

In [0]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [16]:
subword(1000)

"▁i ▁can ' t ▁believe ▁i ▁actually ▁watched ▁this ▁but ▁i ▁guess ▁i ▁just ▁want ed ▁to ▁know ▁if ▁this ▁movie ▁would ▁get ▁un in tention ally ▁fun ni er ▁and ▁fun ni er ▁to ward s ▁the ▁end ."

In [17]:
subword(200)

"▁ i ▁ca n ' t ▁be li e ve ▁ i ▁ act u al ly ▁w at ch ed ▁this ▁but ▁ i ▁g u es s ▁ i ▁ j us t ▁w an t ed ▁to"

### Numericalization with fastai

In [18]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#276) ['xxbos','i','ca',"n't",'believe','i','actually','watched','this','but','i','guess','i','just','wanted','to','know','if','this','movie','would','get','unintentionally','funnier','and','funnier','towards','the','end','.','and'...]


In [19]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#276) ['xxbos','i','ca',"n't",'believe','i','actually','watched','this','but'...]

In [20]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2064) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','in','it','i'...]"

In [21]:
nums = num(toks)[:20]; nums

tensor([  2,  19, 215,  32, 270,  19, 170, 315,  21,  28,  19, 338,  19,  55, 873,  15, 148,  62,  21,  25])

In [22]:
' '.join(num.vocab[o] for o in nums)

"xxbos i ca n't believe i actually watched this but i guess i just wanted to know if this movie"

### Putting our texts into batches for a language model

In [23]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [24]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [25]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [26]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [0]:
nums200 = toks200.map(num)

In [0]:
dl = LMDataLoader(nums200)

In [29]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [30]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos i ca n't believe i actually watched this but i guess i just wanted to know if this movie"

In [31]:
' '.join(num.vocab[o] for o in y[0][:20])

"i ca n't believe i actually watched this but i guess i just wanted to know if this movie would"

## Training a text classifier

### Language model using DataBlock

In [32]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=96, seq_len=80)

In [33]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj you can find an anti - war statement here without looking too hard ; that layer is hackneyed . xxmaj or you can find a value neutral comment on the madness of war ( stripped of "" judgement "" ) ; that layer is completely uninteresting . \n\n xxmaj or you can watch this for the darn good entertainment value of xxmaj duvall 's one - liners , but that 's just a coating for commercial xxunk .","xxmaj you can find an anti - war statement here without looking too hard ; that layer is hackneyed . xxmaj or you can find a value neutral comment on the madness of war ( stripped of "" judgement "" ) ; that layer is completely uninteresting . \n\n xxmaj or you can watch this for the darn good entertainment value of xxmaj duvall 's one - liners , but that 's just a coating for commercial xxunk . \n\n"
1,", as a child had always envied his brother and then one day , things took a turn for the worse when his father gave his entire wealth to xxmaj ram xxmaj bajaj . xxmaj teja decided to kill his father but in the night 's darkness , instead killed xxmaj munshi xxmaj harish xxmaj chandra ( raveena 's xxmaj dad ) , though xxmaj teja does n't regret his actions . xxmaj teja 's sole mission , now ,","as a child had always envied his brother and then one day , things took a turn for the worse when his father gave his entire wealth to xxmaj ram xxmaj bajaj . xxmaj teja decided to kill his father but in the night 's darkness , instead killed xxmaj munshi xxmaj harish xxmaj chandra ( raveena 's xxmaj dad ) , though xxmaj teja does n't regret his actions . xxmaj teja 's sole mission , now , is"


### Fine tuning the language model

In [34]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [35]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.118361,3.919613,0.299835,50.380951,25:33


### Saving and loading models

In [0]:
learn.save('1epoch')

In [38]:
%pwd

'/content/drive/My Drive/fastbook'

In [0]:
learn = learn.load('1epoch')

In [40]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.874949,3.773629,0.317914,43.537796,28:00
1,3.779639,3.721187,0.324357,41.313408,27:59
2,3.730196,3.669733,0.329992,39.241440,28:01
3,3.669103,3.638606,0.333094,38.038769,28:01
4,3.601516,3.614755,0.336278,37.142242,27:58
5,3.529677,3.597694,0.338611,36.513943,27:57
6,3.476085,3.586018,0.340474,36.090080,27:57
7,3.418420,3.583690,0.341267,36.006165,27:56
8,3.366079,3.587576,0.341229,36.146366,27:59
9,3.317522,3.591944,0.340975,36.304592,28:20


In [0]:
learn.save_encoder('finetuned')

### Text generation

In [42]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [43]:
print("\n".join(preds))

i liked this movie because it is one of my all time favorite movies . It is a must watch for any fan of Tom Arnold and Arnold Schwarzenegger . The movie is about an ex - union soldier
i liked this movie because i like Jean Reno . i was n't expecting much when he did this movie . i had heard it was good but it got sillier as he went along . The movie looked great and there


### Creating the classifier DataLoaders

In [0]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [45]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj by now you 've probably heard a bit about the new xxmaj disney dub of xxmaj miyazaki 's classic film , xxmaj laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky . xxmaj during late summer of 1998 , xxmaj disney released "" kiki 's xxmaj delivery xxmaj service "" on video which included a preview of the xxmaj laputa dub saying it was due out in "" 1 xxrep 3 9 "" . xxmaj it 's obviously way past that year now , but the dub has been finally completed . xxmaj and it 's not "" laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky "" , just "" castle xxmaj in xxmaj the xxmaj sky "" for the dub , since xxmaj laputa is not such a nice word in xxmaj spanish ( even though they use the word xxmaj laputa many times",pos
2,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos


In [0]:
nums_samp = toks200[:10].map(num)

In [47]:
nums_samp.map(len)

(#10) [276,274,537,100,483,143,187,184,186,211]

In [0]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [0]:
learn = learn.load_encoder('finetuned')

### Fine tuning the classifier

In [50]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.342431,0.183062,0.929760,01:38


In [51]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.250609,0.168627,0.936080,01:54


In [52]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.192222,0.153499,0.944000,02:41


In [53]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.154143,0.150506,0.946160,03:16
1,0.138245,0.151740,0.944440,03:13
